In [1]:
import torch 
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from random import randint

from PIL import Image,ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
    transforms.RandomRotation(randint(1,360)),
    transforms.RandomHorizontalFlip(),

])

In [3]:
train_path = "../Remote sensing paper/Final Dataset/train"
test_path = "../Remote sensing paper/Final Dataset/test"
val_path = "../Remote sensing paper/Final Dataset/val"

In [4]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [5]:
train_data = torchvision.datasets.ImageFolder(root=train_path,transform=transforms,is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path,transform=transforms,is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path,transform=transforms,is_valid_file=check_image)

In [ ]:
batch_size = 64
num_classes = 10

In [ ]:
train_loader = DataLoader(dataset=train_data,batch_size=batch_size)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
val_loader = DataLoader(dataset=val_data,batch_size=batch_size)

In [ ]:
if torch.cuda.is_available:
    device = torch.device('cuda')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')

device = 'cpu'

In [ ]:
class convblock(nn.Module):

    def __init__(self,in_channels,out_channels,kernel_size,stride,padding=0):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,stride=stride,padding=padding),
            nn.BatchNorm2d(out_channels,eps=0.001),
            nn.ReLU(inplace=True)
        )
    
    def forward(self,x):
        return self.conv(x)


In [ ]:
class aux(nn.Module):

    def __init__(self,in_channels,num_classes):
        super().__init__()
        self.aux_start = nn.Sequential(
            nn.AvgPool2d(kernel_size=(4,4),stride=1),
            convblock(in_channels=in_channels,out_channels=128,kernel_size=(1,1),stride=3),
        )
        self.aux_linear = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.7),
            nn.Linear(1024, num_classes)
        )
    
    def forward(self,x):
        x = self.aux_start(x)
        x = torch.flatten(x,1)
        x = self.aux_linear(x)
        return x
        

In [ ]:
class inception(nn.Module):
    
    def __init__(self,in_channels,f1x1,in3x3,f3x3,in5x5,f5x5,fpool):
        super().__init__()
        self.branch1 = convblock(in_channels=in_channels,out_channels=f1x1,kernel_size=(1,1),stride=1)
        self.branch2 = nn.Sequential(
            convblock(in_channels=in_channels,out_channels=in3x3,kernel_size=(1,1),stride=1),
            convblock(in_channels=in3x3,out_channels=f3x3,kernel_size=(3,3),stride=1,padding=1),
        )
        self.branch3 = nn.Sequential(
            convblock(in_channels=in_channels,out_channels=in5x5,kernel_size=(1,1),stride=1),
            convblock(in_channels=in5x5,out_channels=f5x5,kernel_size=(5,5),stride=1,padding=2),
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(3,3),stride=1,padding=1,ceil_mode=True),
            convblock(in_channels=in_channels,out_channels=fpool,kernel_size=(1,1),stride=1)
        )
    
    def forward(self,x):
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)

        x = torch.cat([x1,x2,x3,x4],1)
        return x

In [ ]:
class Googlenet(nn.Module):

    def __init__(self,in_channels,num_classes,aux_avai=False):

        super().__init__()

        self.conv1 = convblock(in_channels=in_channels,out_channels=64,kernel_size=(7,7),stride=2,padding=3)
        self.maxpool = nn.MaxPool2d(kernel_size=(3,3), stride=2,ceil_mode=True)
        self.conv2 = convblock(in_channels=64,out_channels=64,kernel_size=(1,1),stride=1)
        self.conv3 = convblock(in_channels=64,out_channels=192,kernel_size=(3,3),stride=1,padding=1)

        self.inception3a = inception(in_channels=192,f1x1=64,in3x3=96,f3x3=128,in5x5=16,f5x5=32,fpool=32)
        self.inception3b = inception(in_channels=256,f1x1=128,in3x3=128,f3x3=192,in5x5=32,f5x5=96,fpool=64)

        self.inception4a = inception(in_channels=480,f1x1=192,in3x3=96,f3x3=208,in5x5=16,f5x5=48,fpool=64)
        self.inception4b = inception(in_channels=512,f1x1=160,in3x3=112,f3x3=224,in5x5=24,f5x5=64,fpool=64)
        self.inception4c = inception(in_channels=512,f1x1=128,in3x3=128,f3x3=256,in5x5=24,f5x5=64,fpool=64)
        self.inception4d = inception(in_channels=512,f1x1=112,in3x3=144,f3x3=288,in5x5=32,f5x5=64,fpool=64)
        self.inception4e = inception(in_channels=528,f1x1=256,in3x3=160,f3x3=320,in5x5=32,f5x5=128,fpool=128)


        self.inception5a = inception(in_channels=832,f1x1=256,in3x3=160,f3x3=320,in5x5=32,f5x5=128,fpool=128)
        self.inception5b = inception(in_channels=832,f1x1=384,in3x3=192,f3x3=384,in5x5=48,f5x5=128,fpool=128)

        self.aux_avai = aux_avai
        self.aux4a = aux(in_channels=512,num_classes=num_classes) 
        self.aux4d = aux(in_channels=528,num_classes=num_classes) 

        self.avgpool = nn.AvgPool2d(kernel_size=(7,7),stride=1)

        self.drop = nn.Dropout(0.4)

        self.linear = nn.Linear(in_features=1024,out_features=num_classes)

    def forward(self,x):
        y = None
        z = None
        # print(x.shape)
        x = self.conv1(x)
        # print(x.shape)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.conv2(x)
        # print(x.shape)
        x = self.conv3(x)
        # print(x.shape)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.inception3a(x)
        # print(x.shape)
        x = self.inception3b(x)
        # print(x.shape)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.inception4a(x)
        # print(x.shape)
        
        if self.training and self.aux_avai:
            y = self.aux4a(x)

        x = self.inception4b(x) 
        # print(x.shape)
        x = self.inception4c(x) 
        # print(x.shape)
        x = self.inception4d(x)
        # print(x.shape)

        if self.training and self.aux_avai:
            z = self.aux4d(x) 

        x = self.inception4e(x)
        # print(x.shape)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.inception5a(x)
        # print(x.shape)
        x = self.inception5b(x)
        # print(x.shape)
        x = self.avgpool(x)
        # print(x.shape)
        x = torch.flatten(x,1)
        # print(x.shape)
        x = self.drop(x)
        # print(x.shape)
        x = self.linear(x)
        # print(x.shape)
        return x,y,z


In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=2, aux_avai=False, device=device):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output,aux1,aux2 = model(inputs)
            if aux_avai:
                loss = loss_fn(output, targets) + 0.3*loss_fn(aux1, targets) + 0.3*loss_fn(aux2, targets)
            else:
                loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output,_,_ = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=False)

model.to(device)

In [ ]:
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.0001,weight_decay=1e-4)

loss = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

In [ ]:
train(model=model,optimizer=optimizer,loss_fn=loss,train_loader=train_loader,val_loader=val_loader,epochs=1,aux_avai=True,device=device)